# **FLOW-VIS 3D APPLICATION - SOFTWARE DEVELOPMENT**

*An .ipynb file for flow-vis data representation and quantitative analysis -  application development*

In [5]:
#import statements
from components.user_interface import UserInterface
from components.point_cloud_visualizer import PointCloudVisualizer
from components.point_cloud_registration import PointCloudRegistration
from components.point_cloud_utils import load_pcd

**File selection**

- Select a file to be processed from the data folder.

In [6]:
# Create an instance of UserInterface to handle file selection
ui_pcd = UserInterface()
ui_pcd.select_file()
if ui_pcd.selected_file_path:
    print("Selected file path:", ui_pcd.selected_file_path)
else:
    print("Please select a file")

Selected file path: D:/flowvis_data/RearWing_WT_2343_Einstar_registered_CFD.ply


**Note:** 
At first-time cell execution, the `flowvis_data` folder where the .ply files are stored may not always pop up in the foreground of the Visual Studio Code editor, but in the background or on the side.

#### FLOW-VIS 3D: APPLICATION WORKFLOW

**1. DATA VISUALIZATION**

- Load the 3D point cloud data
- Obtain an interactive representation of the registered point cloud file (.ply) using the `PointCloudVisualizer` class for visualization.

In [7]:
pcd = load_pcd(ui_pcd.selected_file_path)
visualizer = PointCloudVisualizer(input_clouds=pcd)

Button(description='Show 3D data', style=ButtonStyle(button_color='red', font_weight='bold'))

Output()

**2. POINT CLOUD REGISTRATION**

**Target file selection**

* Select the target file for the registration.

In [8]:
# Create an instance of UserInterface to handle file selection
ui_target = UserInterface()
ui_target.select_file()
if ui_target.selected_file_path:
    print("Selected file path:", ui_target.selected_file_path)
    target_path = ui_target.selected_file_path
else:
    print("Please select a file")

Selected file path: D:/flowvis_data/registration/test.ply



* **Objective**: Precise .PLY point cloud registration to CFD car coordinates or wind tunnel car coordinates (UUT), depending on the registration input file. <p>
* **Step 1: Initialization**
    * Initialize the `PointCloudRegistration` class with point cloud data (source and target point clouds), either as a file path or pre-loaded point cloud.<p>
* **Step 2: `registration.register()`**
    * **Operation 1: Feature Detection and Global Registration**
        * The source point cloud is heavily downsampled through and instance of `PointCloudPreprocessor` (no point cloud normals are calculated in this context), and *Fast Point Feature Histograms (FPFH)* are computed.
        * A *RANSAC*-based global registration aligns the point clouds approximately: multiple iterations are performed with different voxel sizes (best values based on system scale and design iterations) to achieve desired accuracy.
        * The target is to achieve a *RANSAC* fitness threshold of 0.90 for high-quality alignment.
        * If the desired *RANSAC* quality is met, we proceed to the next step; otherwise, the user is prompted to repeat registration.   
    * **Operation 2: Fine Alignment with ICP**
        * Point-to-Plane Iterative Closest Point (ICP) is used for fine-grained alignment.
        * User-defined ICP required fitness for each one of the feature scales tested (i.e., [20.0. 25.0, 30.0], heuristic values found through extensive experimentation)
        * The final transformation matrix, metrics, and the registered point cloud are recorded.
        * A visual representation of the registered point cloud (source and target) is enabled for user evaluation by instancing the `PointCloudVisualizer` class.<p>

**Notes:** 
1. There is a chance that alignment may fail on the first try due to bad choice in points sampling during RANSAC global registration. The points are selected randomly by default. To achieve accurate results, repeat the registration process.
2. When registering suspension parts, please upload the .stl file with the corresponding setup conditions if available 

In [9]:
# Create an instance of the PointCloudRegistration class
registration = PointCloudRegistration(source=pcd, target=ui_target.selected_file_path)
# Register the source point cloud to the target point cloud
pcd_registered, transformation, log_text = registration.register(desired_fitness_ransac=0.85, desired_fitness_icp=[0.65, 0.75, 0.85])
#pcd_registered, transformation = registration.register( voxel_sizes=[20.0, 25.0, 30.0, 35.0], desired_fitness_ransac=0.45, desired_fitness_icp=[0.45, 0.45, 0.45, 0.45])

Button(description='Show 3D data', style=ButtonStyle(button_color='red', font_weight='bold'))

Output()

ValueError: too many values to unpack (expected 2)

* **Step 3: `registration.save_registered_ply()`**
    * The registered point cloud is saved as `originalfilename_registered.ply` in the same folder as the original file.
    * The registered scaled point cloud is saved as `originalfilename_registered_paraview.ply` in the same folder as the original file. This file is intended for correlation with CFD in Paraview.
    * If requested (`save_mesh =True`), the registered mesh .ply file is also saved as `originalfilename_registered_mesh_paraview.ply`. This file can be readily uploaded as a mesh in Sandbox for user inspection. The mesh has been registered by applying the transformation matrix found above and two scaling operations are carried out:
        * 1. Scale from mm to m: 1/1000 
        * 2. Scale from MS to FS : 1000/600 <p>

In [ ]:
# Save the registered point cloud to a .ply file in the same location as the priginal file (format "xxx_registered.ply")
registration.save_registered_ply(ui_pcd.selected_file_path, save_mesh=True)

* **Conclusion**: The process provides robust and precise registration to align .PLY point clouds to car coordinates. The registration procedure log is stored in the file `registration_log.txt` in the folder `D:\flowvis_data\registration`.<p>

In [1]:
import open3d as o3d
import open3d as o3d
import tempfile
import os

def stl_to_point_cloud(stl_file_path):
    """
    Converts an STL file to a Point Cloud and visualizes it.

    Args:
        stl_file_path (str): Path to the STL file.

    Returns:
        o3d.geometry.PointCloud: The converted point cloud.
    """
    # Read the mesh from an STL file
    mesh = o3d.io.read_triangle_mesh(stl_file_path)

    # Create a temporary directory to save the intermediate PLY file
    with tempfile.TemporaryDirectory() as temp_dir:
        temp_ply_path = os.path.join(temp_dir, 'temp_mesh.ply')
        
        # Save the mesh to a PLY file
        o3d.io.write_triangle_mesh(temp_ply_path, mesh)
        
        # Read the PLY file back in as a point cloud
        pcd = o3d.io.read_point_cloud(temp_ply_path)
        
        # Optionally visualize the point cloud
        o3d.visualization.draw_geometries([pcd], window_name="Converted Point Cloud from STL")
        
        # No need to manually delete the temp file; it will be removed with the temporary directory
        return pcd

# Example usage
stl_file_path = 'D:/flowvis_data/registration/BODY_M42_Mono_CAD_CFD_MS_v2.stl'
ply_file_path = 'D:/flowvis_data/registration/test.ply'
point_cloud = stl_to_point_cloud(stl_file_path)



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
print(len(point_cloud.points))

12527806


In [4]:
test = o3d.io.read_point_cloud('D:/flowvis_data/registration/BODY_M42_Mono_CAD_CFD_MS_v2.ply')
print(len(test.points))

2450690
